In [17]:
import requests
from requests.exceptions import HTTPError
import json
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


url_main = 'http://127.0.0.1:8000/lta/'
while True:
    response = requests.get(url_main)
    time.sleep(1)
    if response.text != "":
        res_json = response.json()
        print(res_json)
        self_id = res_json["self_id"]
        departure_country = res_json["departure_country"]
        departure_city = res_json["departure_city"]
        date_of_departure = res_json["date_of_departure"]
        changing_the_departure_date = res_json["changing_the_departure_date"]
        destination_country = res_json["destination_country"]
        destination_resort = res_json["destination_resort"]
        return_date = res_json["return_date"]
        changing_return_date = res_json["changing_return_date"]
        number_of_adults = res_json["number_of_adults"]
        number_of_children = res_json["number_of_children"]
        number_of_infants = res_json["number_of_infants"]
        direct_flights_only = res_json["direct_flights_only"]

       
        if departure_city == "Москва":
            departure_city = "MOW"
        destination_resort = "KZN"
        sp_date_of_departure = str(date_of_departure).split('-')
        date_of_departure = sp_date_of_departure[2] + sp_date_of_departure[1]
        sp_return_date = str(return_date).split('-')
        return_date = sp_return_date[2] + sp_return_date[1]

        url = 'https://www.aviasales.ru/search/' + departure_city + date_of_departure + destination_resort + return_date + number_of_adults + number_of_children + number_of_infants
        while url[-1] == '0':  # отбрасываем нули из конца запроса, чтобы он был похож на настоящий :))
            url = url[:-1]
        chrome_options = webdriver.ChromeOptions()
        chrome_options.add_argument("--incognito")
        #chrome_options.add_argument('--headless')
        driver = webdriver.Chrome(
            options=chrome_options, executable_path="C:/PyThon/chromedriver.exe")
        print(url)
        driver.get(url)
        #time.sleep(40)
        # Определение конца загрузки сайта ЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖ
        try:
            WebDriverWait(driver, 50).until(
                EC.presence_of_element_located((By.XPATH, "//*[@class='loader__bar --animation-finished']")))
        except Exception:
            print('Ошибка - страница загружена не полностью. Возможно входные данные недостоверны')
        else:
            print('ОК - Страница загрузилась полностью')
        # time.sleep(1)
        # Поиск цен по разному кол-ву пересадок ЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖ
        alt1 = ''
        src1 = ''
        alt2 = ''
        src2 = ''
        price_0_transfer = ''
        price_1_transfer = ''
        price_2_transfer = ''

        tt = driver.page_source
        price_0_transfer = "Не найдено"
        price_1_transfer = "Не найдено"
        price_2_transfer = "Не найдено"
        f = tt.split('class="checkboxes-list__extra-uncheck-other')
        if len(f) == 2:
            f1 = f[1].split('span class="price --rub">')
            price_0_transfer = f1[1].split("<")[0].replace('\u2009', '')
        if len(f) == 3:
            f1 = f[1].split('span class="price --rub">')
            f2 = f[2].split('span class="price --rub">')
            price_0_transfer = f1[1].split("<")[0].replace('\u2009', '')
            price_1_transfer = f2[1].split("<")[0].replace('\u2009', '')
        if len(f) > 3:
            f1 = f[1].split('span class="price --rub">')
            f2 = f[2].split('span class="price --rub">')
            f3 = f[3].split('span class="price --rub">')
            price_0_transfer = f1[1].split("<")[0].replace('\u2009', '')
            price_1_transfer = f2[1].split("<")[0].replace('\u2009', '')
            price_2_transfer = f3[1].split("<")[0].replace('\u2009', '')
        # Поиск самой дешевой цены ЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖ
        v = []
        lowest_price = ""
        xe = driver.find_element_by_xpath("//*[@class='product-list__item fade-enter-done']")
        xe2 = list(map(lambda x: x.replace('\u2009', ''), xe.text.split('\n')))
        try:
            lowest_price = str(xe2[xe2.index("Купить") + 1][3:])
        except ValueError:
            print('Ошибка - Не найдена самая низка цена из 10-ти запросов')
        # Поиск времени в пути ЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖ
        for i in xe2:
            if i.startswith('В пути'):
                v.append(i)
        if not v:
            v = ['-', '-']
            print('Ошибка - Не найдено время в пути')
        luggage = xe2[0]  # тут определяем багаж
        # Поиск названия авиакомпании и ссылки на картинку ЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖЖ
        try:
            xe4 = driver.find_elements_by_xpath(
                "//*[@class='product-list__item fade-enter-done'][1]//*[@class='ticket-carrier__img']")
        except ValueError:
            print('Ошибка - Не найдено название авиакомпании')
        else:
            attrs = []
            if len(xe4) == 1:
                for attr in xe4[0].get_property('attributes'):
                    attrs.append([attr['name'], attr['value']])
                    for i in range(len(attrs)):
                        if attrs[i][0] == 'src':
                            src1 = attrs[i][1][2:]
                        if attrs[i][0] == 'alt':
                            alt1 = attrs[i][1]
            else:
                for attr in xe4[0].get_property('attributes'):
                    attrs.append([attr['name'], attr['value']])
                    for i in range(len(attrs)):
                        if attrs[i][0] == 'src':
                            src1 = attrs[i][1][2:]
                        if attrs[i][0] == 'alt':
                            alt1 = attrs[i][1]
                for attr in xe4[1].get_property('attributes'):
                    attrs.append([attr['name'], attr['value']])
                    for i in range(len(attrs)):
                        if attrs[i][0] == 'src':
                            src2 = attrs[i][1][2:]
                        if attrs[i][0] == 'alt':
                            alt2 = attrs[i][1]
            print('ОК - Найдено авиакомпаний:', len(xe4))

        rezult = {"self_id": self_id,
          "price_0_transfer": price_0_transfer,
          "price_1_transfer": price_1_transfer,
          "price_2_transfer": price_2_transfer,
          "lowest_price": lowest_price,
          "luggage": luggage,
          "time_there": v[0].lower(),
          "time_back": v[1].lower(),
          "airline1_name": alt1,
          "link1": src1,
          "airline2_name": alt2,
          "link2": src2,
         }
        requests.post('http://127.0.0.1:8000/lta/', json=rezult)
        driver.quit()
        print("FINISH lta", self_id)
        print("#######################################################################################################")
    else:
        time.sleep(15)


{'self_id': 12, 'departure_country': 'RU', 'departure_city': 'Киев', 'date_of_departure': '2020-08-01', 'changing_the_departure_date': 'var_day_0', 'destination_country': 'Таиланд', 'destination_resort': 'Самуи', 'hotel_name': '', 'count_of_hotel_stars': '0', 'hotel_rating': '0', 'type_of_food': '0', 'return_date': '2020-11-07', 'changing_return_date': 'var_day_0', 'number_of_adults': '2', 'number_of_children': '0', 'number_of_infants': '0', 'direct_flights_only': 'False', 'accommodation_at_the_hotel_only': 'False'}


KeyboardInterrupt: 